In [1]:
# Standart Libraries

import numpy as np

# Import files

import utilities
from Filter import ExtendedKalmanFilter
from Radar import Radar
from RadarSystem import RadarSystem
from Earth import Earth
from Satellite import Satellite
from SatelliteState import SatelliteState

from copy import deepcopy
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
import random

from config import config



In [2]:
class Agent():
    def __init__(self, cov_0, v_0):
        self.cov = cov_0
        self.v = np.diag(v_0)
        
        self.pbest = self.get_performance()
        self.pbest_cov = self.cov
    
    def update_performance(self):
        #THIS ASSUMES SELF.COV HAS BEEN CHANGED
        new_perf = self.get_performance()
        if new_perf < self.pbest:
            self.pbest = new_perf
            self.pbest_cov = self.cov

    def get_performance(self):
        perf = []
        for i in range(5):
            perf.append(self.get_val())
        return np.mean(perf)
            
    def get_val(self):

        # Initialize Earth
        earth = Earth()

        # Initialize RadarSystem
        # Beijing Aerospace Command and Control Center

        BACC =  RadarSystem(500, Earth(), seed=27) 

        # Initialize Satellite
        R = config['satellite']['initial_conditions']['distance']
        theta = config['satellite']['initial_conditions']['polar_angle']
        phi = config['satellite']['initial_conditions']['azimuthal_angle']

        angular_vel = 0.0010830807404
        tang_vel = angular_vel * R
        radial_velocity = 0
        azimuthal_velocity = 0


        sat_state = SatelliteState(np.array([R, theta, phi]), np.array([0]), np.array([radial_velocity, tang_vel, azimuthal_velocity]), np.array([0]))
        tiagong = Satellite(sat_state, 0, earth=earth)

        simulation = tiagong.simulate(10000000)

        sim_lenght = len(simulation.y[0])
        R_sim = simulation.y[0][:]-6378136.6
        rad_sim = simulation.y[2][:]

        sim_values = [R_sim, rad_sim]
        sim_values = np.reshape(sim_values, [2, sim_lenght])

        r_noise = config['radar']['noise']['rho']
        t_noise = config['radar']['noise']['theta']
        mean_0 = np.array([R+np.random.normal(0,r_noise), 0, 0, (np.pi/2)+np.random.normal(0,t_noise), 0, 0])
 
        observation_noise = np.array([[10e2, 0],
                                      [0, 10e-5]])

        Q = np.array([
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0.0, 0],
            [0, 0, 0, 0, 0, 0.0]
        ])

        tianhe = ExtendedKalmanFilter(mean_0, self.cov, earth, observation_noise=observation_noise, process_noise=Q)

        m = deepcopy(mean_0)
        m = np.array([m[0], m[3]])
        predicted_states_satellite_cord = [m]
        radar_states_satellite_cord = [m]
        for i in range(int(sim_lenght)-1):
            
            if i < sim_lenght:
                current_state_satellite_cord = tiagong.get_position_at_t(i)
                current_state_earth_cord = utilities.spherical_to_spherical(current_state_satellite_cord)
                noise_states_earth_cord = BACC.try_detect_satellite(current_state_earth_cord, i)

                if len(noise_states_earth_cord) > 0:
                    flag = 0
                    for state_earth_cord in noise_states_earth_cord:
                        state_satellite_cord = utilities.spherical_to_spherical(state_earth_cord.pos)
                        new_state_satellite_cord = tianhe.update(state_satellite_cord[:2])

                        if flag == 0:
                            radar_states_satellite_cord += state_satellite_cord[:2],
                            flag = 1
                    
            forecast = tianhe.forecast()
            new_state_satellite_cord = [forecast[0][0][0], forecast[0][3][0]]
            predicted_states_satellite_cord += new_state_satellite_cord,


        '''
        Here we obtain RMSE, however we dont want to it to the larger R as such we do a further calc
        '''
        R2, rad2 = np.array(predicted_states_satellite_cord[:]).T
        R2 = R2 - 6378136.6
        k_values = [R2, rad2]
        k_values = np.reshape(k_values, [2, sim_lenght])
        squared_diff = (sim_values - k_values) ** 2
        mse = np.mean(squared_diff, axis=1)
        rmse = np.sqrt(mse)

        '''
        Out obj_func
        '''
        val = rmse[0] * 0.5 + rmse[1] * 1000

        return val


In [3]:
cov_0 = np.array([
    [10e5, 0, 0, 0, 0, 0],
    [0, 10, 0, 0, 0, 0],
    [0, 0, 10, 0, 0, 0],
    [0, 0, 0, 1.5, 0, 0],
    [0, 0, 0, 0, 0.5, 0],
    [0, 0, 0, 0, 0, 0.5]
])

v_0 = [-100, -1, -1, -0.1, -0.03, -0.1]

cov_1 = np.array([
    [10e6, 0, 0, 0, 0, 0],
    [0, 20, 0, 0, 0, 0],
    [0, 0, 10, 0, 0, 0],
    [0, 0, 0, 5, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 3]
])

v_1 = [-1000, -3, -1, -0.1, -0.03, -0.4]

cov_2 = np.array([
    [10e9, 0, 0, 0, 0, 0],
    [0, 20, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 0],
    [0, 0, 0, 5, 0, 0],
    [0, 0, 0, 0, 0.001, 0],
    [0, 0, 0, 0, 0, 3]
])

v_2 = [-270, -3, 0.1 , -0.1, 0.001, 0.1]

agents = {}
agents[0] = Agent(cov_0 = cov_0, v_0 = v_0)
agents[1] = Agent(cov_0 = cov_1, v_0 = v_1)
agents[2] = Agent(cov_0 = cov_2, v_0 = v_2)


In [4]:
print(agents[0].v)

[[-1.e+02  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00 -1.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00 -1.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00 -1.e-01  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00 -3.e-02  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-01]]


In [5]:
gbest = 100000
gbest_cov = cov_0
W = 0.3 #How much you want to stay on the velocity.

#Exploration over exploitation
c1 = 0.3
c2 = 0.7

for i in range(10):
  
    #GETS GBEST
    for i in range(len(agents)):
        agent = agents[i]
        if agent.pbest < gbest:
            gbest = agent.pbest
            gbest_cov = agent.pbest_cov

    #COVARIANCE UPDATE ON VELOCITY
    for i in range(len(agents)):
        agent = agents[i]
        agent.v = W*agent.v + c1*random.random()*(agent.pbest_cov - agent.cov) + c2*random.random()*(gbest_cov - agent.cov)
        agent.cov = agent.cov + agent.v
        agent.update_performance()

In [8]:
print(gbest_cov)

[[9.99989323e+05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.89323398e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.89323398e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.48932340e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.96797019e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.89323398e-01]]
